# Exact solution

Ethier C.R., Steinman D.A., Exact fully 3d Navier-Stokes solutions for benchmarking


We test here for a stationary solution (d_const = 0). Any mesh should work.

In [1]:
from dolfin import *
import numpy as np
import mshr as mshr

# Should stay 1 here (as we used normalized equation)
mu = Constant(1.)
muv = mu.values()[0]

a_const = 2.
d_const = 1.

outerR = 1.
h = 2.

outerCyl = mshr.Cylinder(Point([0.,0.,0.]),Point([0.,0.,h]),outerR,outerR)

In [2]:
# We mark physical boundary and setup the space of finite elements 
mark = {"generic": 0,
        "wall": 1}

class Bondary_w(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary 

wall = Bondary_w()

#File("bord.pvd") << subdomains

degree = 2
elemf0 = FiniteElement('P-', cell='tetrahedron',form_degree=0, degree=degree)
elemf1 = FiniteElement('P-', cell='tetrahedron',form_degree=1, degree=degree)
elemf2 = FiniteElement('P-', cell='tetrahedron',form_degree=2, degree=degree)
elemf3 = FiniteElement('P-', cell='tetrahedron',form_degree=3, degree=degree)
elemH = FiniteElement('Real',cell='tetrahedron',degree=0)

TH = MixedElement([elemf1,elemf2,elemf3])

In [3]:
# We define physical parameters

nbitt = 100
#T = 8.0
#timestep = Constant(T/nbitt)
#print(timestep.values()[0])
timestep = Constant(0.01)

noslip = Constant((0.,0.,0.))
force = Constant((0.,0.,0.))

u_exactt = Expression(('0.','0.','0.',
                      '-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)','0.'),
                     degree=2, a = a_const,d = d_const,t = 0.)

h = []
err = []

In [4]:
# First loop
mesh=mshr.generate_mesh(outerCyl,15)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)



# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

88467
t = 0.000
0.20589834944615557
t = 0.010
0.9304092720611445
t = 0.020
0.9165347821386044
t = 0.030
0.9038828206382805
t = 0.040
0.8859433983568153
t = 0.050
0.8682572462503253
t = 0.060
0.851115052650186
t = 0.070
0.8341502895168884
t = 0.080
0.8178501097411007
t = 0.090
0.8018368214603101
t = 0.100
0.7859752766243133
t = 0.110
0.7704892733968866
t = 0.120
0.7553084079482866
t = 0.130
0.7403599184976416
t = 0.140
0.7257207360528759
t = 0.150
0.7113830660698447
t = 0.160
0.6973006120176296
t = 0.170
0.6834975985761175
t = 0.180
0.6699771579162189
t = 0.190
0.6567116487629986
t = 0.200
0.6437079441711614
t = 0.210
0.6309675704047064
t = 0.220
0.6184731653450153
t = 0.230
0.6062253153993666
t = 0.240
0.5942236530233652
t = 0.250
0.5824565990787831
t = 0.260
0.5709216639647638
t = 0.270
0.5596172970589698
t = 0.280
0.548535564954344
t = 0.290
0.537672497427791
t = 0.300
0.527025636185455
t = 0.310
0.5165892952028645
t = 0.320
0.5063590962027475
t = 0.330
0.4963319309135819
t = 0.340
0

In [5]:
# Second loop
mesh=mshr.generate_mesh(outerCyl,20)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)



# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

199925
t = 0.000
0.06627583574006533
t = 0.010
0.9252143286829333
t = 0.020
0.9162751584284952
t = 0.030
0.9047826798167707
t = 0.040
0.8868750102720977
t = 0.050
0.8694747280312564
t = 0.060
0.852811385912385
t = 0.070
0.8359528565510922
t = 0.080
0.8194303041216784
t = 0.090
0.803272770762413
t = 0.100
0.7873956960610355
t = 0.110
0.7718328169534647
t = 0.120
0.7565738697387386
t = 0.130
0.7416074326957871
t = 0.140
0.7269322427714942
t = 0.150
0.7125505259564278
t = 0.160
0.6984491364987845
t = 0.170
0.684621610404336
t = 0.180
0.6710710930968974
t = 0.190
0.657786851412204
t = 0.200
0.6447626195512813
t = 0.210
0.6319980729623446
t = 0.220
0.6194853164048059
t = 0.230
0.6072189515564302
t = 0.240
0.5951962949923482
t = 0.250
0.5834112772476264
t = 0.260
0.5718590345451249
t = 0.270
0.560535875736627
t = 0.280
0.5494367758601412
t = 0.290
0.5385572101518787
t = 0.300
0.5278931943612735
t = 0.310
0.5174403020138303
t = 0.320
0.5071942951464982
t = 0.330
0.4971512086847313
t = 0.340
0

In [6]:
# Third loop
mesh=mshr.generate_mesh(outerCyl,25)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)



# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

391448
t = 0.000
0.04332726926381761
t = 0.010
0.9249152562847834
t = 0.020
0.9164194016415101
t = 0.030
0.9051127778718914
t = 0.040
0.8873568302809661
t = 0.050
0.8702424082532106
t = 0.060
0.853660306762246
t = 0.070
0.8367642431659377
t = 0.080
0.8202387266979064
t = 0.090
0.8040726676708841
t = 0.100
0.7881761591105767
t = 0.110
0.772585980176273
t = 0.120
0.7573051767356698
t = 0.130
0.7423286795889474
t = 0.140
0.7276340601760846
t = 0.150
0.7132319198920786
t = 0.160
0.699119398548534
t = 0.170
0.6852772060359288
t = 0.180
0.6717104547739606
t = 0.190
0.6584145978981901
t = 0.200
0.6453776681562361
t = 0.210
0.6325995694362525
t = 0.220
0.6200754122943853
t = 0.230
0.6077974034267273
t = 0.240
0.5957627412445479
t = 0.250
0.5839667553613802
t = 0.260
0.5724036119948887
t = 0.270
0.5610694565012518
t = 0.280
0.5499598968470625
t = 0.290
0.5390700474825044
t = 0.300
0.5283957944511158
t = 0.310
0.5179329799534714
t = 0.320
0.5076772500412214
t = 0.330
0.49762455873785894
t = 0.34

In [7]:
# fourth loop
mesh=mshr.generate_mesh(outerCyl,30)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)

u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = 0.)

# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

668960
t = 0.000
0.026106048402074814
t = 0.010
0.9249370832022853
t = 0.020
0.9172147295542005
t = 0.030
0.905953420129013
t = 0.040
0.8878201870123026
t = 0.050
0.870536857277665
t = 0.060
0.853949414663241
t = 0.070
0.8370876627440355
t = 0.080
0.8205854880901544
t = 0.090
0.8044329799784636
t = 0.100
0.788542873323447
t = 0.110
0.7729469237217351
t = 0.120
0.75766316593941
t = 0.130
0.7426826422837993
t = 0.140
0.727980331264937
t = 0.150
0.713573580678819
t = 0.160
0.6994543736329946
t = 0.170
0.6856048566672739
t = 0.180
0.6720329997397758
t = 0.190
0.6587304033391884
t = 0.200
0.6456868700217183
t = 0.210
0.6329034016931323
t = 0.220
0.620372990467953
t = 0.230
0.6080889910685656
t = 0.240
0.5960489629028459
t = 0.250
0.584247225475678
t = 0.260
0.5726785507364794
t = 0.270
0.5613391973050469
t = 0.280
0.5502243153166704
t = 0.290
0.5393293002737681
t = 0.300
0.5286500861375908
t = 0.310
0.5181823099833305
t = 0.320
0.5079217321155934
t = 0.330
0.49786433671511465
t = 0.340
0.48

In [ ]:
# Fifth loop
mesh=mshr.generate_mesh(outerCyl,35)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)



# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

In [ ]:
# Sixth loop
mesh=mshr.generate_mesh(outerCyl,40)

subdomains = MeshFunction("size_t", mesh,  mesh.topology().dim()-1)
subdomains.set_all(mark["generic"])
wall.mark(subdomains, mark["wall"])

W = FunctionSpace(mesh, TH)



# Define our function for the variationnal problem and our neumanBC (required TestFunctions to be already defined)
w_old = Function(W)
w_old.interpolate(u_exactt)
(u1_o, u2_o, u3_o) = split(w_old)
(u1, u2, u3) = TrialFunctions(W)
(v1, v2, v3) = TestFunctions(W)
n = FacetNormal(mesh)

dsW = Measure('ds', domain=mesh, subdomain_id=mark["wall"], subdomain_data=subdomains)



# Variationnal formulation (except for u_t, a and L are multiplied by timestep below)
a1 = dot(u1,v1)*dx - dot(u2,curl(v1))*dx
a2 = mu*dot(curl(u1),v2)*dx + 0.5*dot(cross(u1,u2_o),v2)*dx + 0.5*dot(cross(u1_o,u2),v2)*dx - dot(u3,div(v2))*dx
a3 = dot(div(u2),v3)*dx
a = a1 + a2 + a3

t = 0
w1 = Function(W)
print(W.dim())
for i in range(nbitt):
    print("t = {:.3f}".format(t))
    u_exact = Expression(('-a*(exp(a*x[0])*sin(a*x[1] + d*x[2]) + exp(a*x[2])*cos(a*x[0] + d*x[1]))*exp(-d*d*t)', 
                      '-a*(exp(a*x[1])*sin(a*x[2] + d*x[0]) + exp(a*x[0])*cos(a*x[1] + d*x[2]))*exp(-d*d*t)',
                      '-a*(exp(a*x[2])*sin(a*x[0] + d*x[1]) + exp(a*x[1])*cos(a*x[2] + d*x[0]))*exp(-d*d*t)'), 
                     degree=2, a = a_const,d = d_const,t = t)
    
    bcu_wall = DirichletBC(W.sub(1), u_exact, subdomains, mark["wall"])
    dbcs = [bcu_wall]
    neumanBC = dot(cross(u_exact,v1),n)*ds
    L = dot(force,v2)*dx + neumanBC
    
    Fa = timestep*a + dot(u2,v2)*dx 
    Fl = timestep*L + dot(w_old.sub(1),v2)*dx
    problem = LinearVariationalProblem(Fa,Fl, w1, dbcs)
    solver = LinearVariationalSolver(problem)
    solver.parameters['linear_solver'] = "mumps"
    
    try:
        solver.solve()
    except RuntimeError:
        print("Solver failed")
        break
    
    tmp = assemble(dot(w1.sub(1) - w_old.sub(1),w1.sub(1) - w_old.sub(1))*dx)
    print(tmp)
    

    w_old.assign(w1)
    t = t + timestep.values()[0]

h.append(mesh.hmax())
err.append(assemble(dot(w1.sub(1) - u_exact,w1.sub(1) - u_exact)*dx))
print(h[-1])
print(err[-1])
print(assemble(div(w1.sub(1))*div(w1.sub(1))*dx))

In [8]:
for i in range(1,len(h)):
    print((np.log(err[i]) - np.log(err[i-1]))/(np.log(h[i]) - np.log(h[i-1])))

3.68511848642
3.8191681433
4.35431261054


In [9]:
import math
rate = []
for i in range(1,len(h)):
    # 0.5 as we computed the square of the norm
    rate.append(0.5*(np.log(err[i]) - np.log(err[i-1]))/(np.log(h[i]) - np.log(h[i-1])))

moy = np.mean(rate)
var = 0
for i in range(len(rate)):
    var = var + (rate[i] - moy)**2
var = var/(len(rate)-1)
print(moy)
print(math.sqrt(var)*3.182)

1.97643320671
0.5633164944807799


In [10]:
for i in range(1,len(h)):
    print(h[i])
for i in range(1,len(err)):
    print(err[i])

0.2790259042326482
0.22388798113452063
0.18788780891378218
0.05584148221181682
0.0240875135573267
0.01122764488064492
